In [1]:
import json
all_exs = json.load(open("/mnt/infonas/data/yashgupta/data/new_bs_processed_data/train_top_row_processed.json"))

In [2]:
print(all_exs[0].keys())
print(all_exs[0])


dict_keys(['answer-text', 'context', 'title', 'question', 'question_id', 'is_impossible', 'answers'])
{'answer-text': 'Ghurids', 'context': "Name is Great Mosque of Herat . Province is Herat Province . City is Herat . Year is 1446 . Remarks is The mosque was the city 's first congregational mosque , built on the former site of two smaller Zoroastrian Fire temples destroyed by earthquake and fire . The Great Mosque of Herat ( Persian : مسجد جامع هرات , Masjid-i Jāmi-i Herāt ) or Jami Masjid of Herat , is a mosque in the city of Herat , in the Herat Province of north-western Afghanistan . It was built by the Ghurids , under the rule of the Sultan Ghiyath al-Din Muhammad Ghori , who laid its foundation in 1200 CE , and later extended by several rulers as Herat changed rulers down the centuries from the Timurids to the Safavids , Mughals and then the Uzbeks , all of whom supported the mosque . Though many of the glazed tiles have been replaced during subsequent periods , the Friday Mosque 

In [3]:
import re
import copy
cnt = 0
sa_exs = []
ma_exs = []
for ex in all_exs:
    ans = ex['answers'][0]['text']
    if (ex['context']).count(ans) > 1:
        ocs = [m.start() for m in re.finditer(re.escape(ans), ex['context'])]
        # print(ocs)
        # break
        cnt += 1
        for j, oc in enumerate(ocs):
            exc = copy.deepcopy(ex)
            exc['answers'][0]['answer_start'] = oc
            # print(oc)
            exc['question_id'] += str(j)
            # print(exc)
            ma_exs.append(exc)
        # break
    else:
        sa_exs.append(ex)
print("multi answer exs", cnt) 
print("single answer exs", len(sa_exs))

multi answer exs 19365
single answer exs 36719


In [9]:
json.dump(sa_exs, open("/mnt/infonas/data/yashgupta/data/new_bs_processed_data/train_processed_top_row_no_group_sa.json", "w"), indent=2)
json.dump(ma_exs, open("/mnt/infonas/data/yashgupta/data/new_bs_processed_data/train_processed_top_row_no_group_ma.json", "w"), indent=2)
json.dump(ma_exs[:16], open("/mnt/infonas/data/yashgupta/data/new_bs_processed_data/train_processed_top_row_no_group_ma_toy.json", "w"), indent=2)

In [5]:
ma_scrs = json.load(open("/mnt/infonas/data/yashgupta/save/bert_large_top_no_group_sa/2021_09_04_19_52_05/checkpoint-epoch1/ma_full_loss_logits.json"))
print(len(ma_scrs))

343052


In [30]:
dct = {}
for k, v in ma_scrs.items():
    ks = k.split("_")
    tk = ks[0]
    # print(ks[2])
    if ks[2]!="loss":
        continue
    if tk in dct.keys():
        if dct[tk][0] > v:
            dct[tk] = (v, k)
    else:
        dct[tk] = (v, k)
print(len(dct.keys()))

19365


In [31]:
# dct

In [32]:
st = set([])
for v in dct.values():
    x = "_".join(v[1].split("_")[:2])
    st.add(x.rstrip('#'))
print(len(st))

19365


In [33]:
# print(st)

In [34]:
ch_ma_ex  = []
for ma_ex in ma_exs:
    if ma_ex['question_id'] in st:
        ch_ma_ex.append(ma_ex)
print(len(ch_ma_ex))

19365


In [35]:
tot_exs = ch_ma_ex + sa_exs
print(len(tot_exs))

56084


In [36]:
json.dump(tot_exs, open("/mnt/infonas/data/yashgupta/data/pl_processed_data/train_processed_top_row_no_group_total_loss.json", "w"), indent=2)
json.dump(tot_exs[:25], open("/mnt/infonas/data/yashgupta/data/processed_data/train_processed_top_row_no_group_total_loss_toy.json", "w"), indent=2)

In [37]:
cnt = 0
for chx in ch_ma_ex:
    if (chx['question_id'][-2:]!="00"):
        cnt += 1
print(cnt)

7187


In [10]:
import argparse
import glob
import logging
import os
import sys
import re
import collections
import random
import timeit
import json
import numpy as np
from datetime import datetime
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
from multiprocessing import Pool, cpu_count
from functools import partial
from torch.utils.data import TensorDataset
#TODO: For longformer
#from transformers import LongformerModel, LongformerTokenizer
#from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import LongformerForQuestionAnswering, LongformerTokenizerFast,LongformerTokenizer, EvalPrediction, LongformerConfig
from transformers import RobertaForQuestionAnswering, RobertaTokenizerFast,RobertaTokenizer,RobertaConfig

 
from transformers import (WEIGHTS_NAME, AdamW, BertConfig, BertTokenizer, 
                        BertForQuestionAnswering, get_linear_schedule_with_warmup)

from transformers.data.metrics.squad_metrics import (
    compute_predictions_log_probs,
    compute_predictions_logits,
)
# from compute_predictions_logits import compute_predictions_logits
import string
from transformers.data.processors.utils import DataProcessor
from utils.utils import readGZip

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [12]:
# MODEL_CLASSES = {"bert": (BertConfig, BertForQuestionAnswering, BertTokenizer)
#             ,"longformer":(LongformerConfig, LongformerForQuestionAnswering, LongformerTokenizer)
#             ,"roberta":(RobertaConfig, RobertaForQuestionAnswering, RobertaTokenizer)}
# config_class, model_class, tokenizer_class = MODEL_CLASSES["bert"]
# ckpt_pth = "/mnt/infonas/data/yashgupta/save/bert_large_logits_top_all_sa/2021_08_30_14_12_37/checkpoint-epoch3/"
# tokenizer = tokenizer_class.from_pretrained(ckpt_pth)
# model = model_class.from_pretrained(ckpt_pth)

In [13]:
# device = torch.device("cuda")
# model.to(device)

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 47.54 GiB total capacity; 1.14 GiB already allocated; 5.62 MiB free; 1.14 GiB reserved in total by PyTorch)